In [3]:
#Importing libraries
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, MultiLineString
from descartes import PolygonPatch

import matplotlib.pyplot as plt
import folium

In [109]:
# Specify type of data
tags = {'shop': True}

# Download building geometries from OSM
store_gdf = ox.geometries_from_place('Eugene, Oregon, USA', tags)

In [110]:
stores = store_gdf[store_gdf['shop'] == 'supermarket'].reset_index()
stores = stores.dropna(subset=['addr:street'])
stores = stores.dropna(axis=1, how='all')
stores['centroid'] = stores['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))
stores.head()

,element_type,osmid,addr:housenumber,addr:postcode,addr:street,brand,brand:wikidata,brand:wikipedia,check_date:opening_hours,name,...,delivery,takeaway,nodes,building,name:fa,ref:walmart,roof:levels,branch,delivery:partner,centroid
0,node,768234549,145,97401,East 18th Avenue,Safeway,Q1508234,en:Safeway Inc.,2023-02-17,Safeway,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.09063 44.04073)
3,node,2527503514,748,97405,East 24th Avenue,NaN,NaN,NaN,NaN,Sundance Natural Foods,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.08136 44.03316)
5,node,3198918820,357,97402,Van Buren Street,NaN,NaN,NaN,NaN,Red Barn Natural Grocery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.10924 44.05593)
7,node,6223446460,4275,NaN,"Barger Dr, Eugene, OR 97402, USA",WinCo Foods,Q8023592,en:WinCo Foods,NaN,WinCo Foods,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.16936 44.08644)
10,node,8034746208,1500,97401,Cal Young Road,Safeway,Q1508234,en:Safeway Inc.,NaN,Safeway,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.07044 44.07794)


In [111]:
# Define center of map (i.e. Condon Hall) and initial zoom level
lat_lon = [44.0751, -123.0781]
m = folium.Map(location=lat_lon, zoom_start=12)

for i in range(0, stores.shape[0]):
    my_string = stores.iloc[i]['name']
    folium.Marker([stores.iloc[i]['centroid'].y, stores.iloc[i]['centroid'].x],
                 popup=my_string).add_to(m)

# Display map
m

In [112]:
# Specify type of data
tags = {'highway': True}

# Download building geometries from OSM
stops_gdf = ox.geometries_from_place('Eugene, Oregon, USA', tags)

In [123]:
stops = stops_gdf[stops_gdf['highway'] == 'bus_stop'].reset_index()
stops = stops.dropna(axis=1, how='all')
stops = stops[stops.geom_type == 'Point']
stops['centroid'] = stops['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))
stops.head()

,element_type,osmid,highway,geometry,ref,source,tactile_paving,name,description,wheelchair,...,route_ref,departures_board,seats,brand,brand:wikidata,local_ref,nodes,area,shelter_type,centroid
0,node,1154028388,bus_stop,POINT (-123.08245 44.04769),NaN,NaN,NaN,Hilyard Station (westbound),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.08245 44.04769)
1,node,1154028392,bus_stop,POINT (-123.06121 44.04479),NaN,NaN,NaN,Walnut Station Inbound,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.06121 44.04479)
2,node,1154028406,bus_stop,POINT (-123.06124 44.04472),NaN,NaN,NaN,Walnut Station Outbound,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.06124 44.04472)
3,node,1154028435,bus_stop,POINT (-123.06934 44.04645),NaN,NaN,NaN,Agate Station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.06934 44.04645)
4,node,1154028449,bus_stop,POINT (-123.06936 44.04638),NaN,NaN,NaN,Agate Station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-123.06936 44.04638)


In [127]:
# Define center of map (i.e. Condon Hall) and initial zoom level
lat_lon = [44.0751, -123.0781]
m = folium.Map(location=lat_lon, zoom_start=12)

for i in range(0, stops.shape[0]):
    my_string = stops.iloc[i]['name']
    folium.Marker([stops.iloc[i]['centroid'].y, stops.iloc[i]['centroid'].x],
                 popup=my_string).add_to(m)

# Display map
m

In [8]:
# Define coordinates of Condon Hall
lat_lon = (44.0451, -123.0781)

# Define walkable street network 3.2 km around Condon Hall
g = ox.graph_from_point(lat_lon, dist=3200, network_type='drive')

In [22]:
# Convert to graph
graph_proj = ox.project_graph(g, to_crs=None)
type(graph_proj)

networkx.classes.multidigraph.MultiDiGraph

In [35]:
stores_proj = stores.to_crs('EPSG:32610')
#nodes = ox.graph_to_gdfs(stores_proj, nodes=True, edges=False)
#edges = ox.graph_to_gdfs(stores_proj, nodes=False, edges=True)
stores_proj.head()

AttributeError: 'GeoDataFrame' object has no attribute 'graph'